# Aircraft Design - SimPleAC

Here, we implement an aircraft design problem proposed by Prof. Warren Hoburg and refined in Berk Ozturk's Master's thesis. See documentation:

* Primary reference is the associated model in [GPLibrary](https://github.com/convexengineering/gplibrary/tree/master/gpkitmodels/SP/SimPleAC)
* Berk's thesis is [available here](https://dspace.mit.edu/handle/1721.1/115595?show=full).

We begin our modeling by initializing an optimization environment:

In [291]:
import aerosandbox as asb
import aerosandbox.numpy as np

opti = asb.Opti()

We continue by defining all of the constants in SimPleAC. For convenience, we define these here all as floats.

If we were interested in studying parametric sensitivities with respect to these constants, we could just as easily define these as parameters using `opti.parameter()` instead, and get the associated duals with `opti.dual(opti.parameter())`.

Note: we have converted all units into base SI units where applicable. (km -> m, etc.)

In [292]:
##### Constants

### Env. constants
g = 9.81  # gravitational acceleration, m/s^2
mu = 1.775e-5  # viscosity of air, kg/m/s
rho = 1.23  # density of air, kg/m^3
rho_f = 817  # density of fuel, kg/m^3

### Non-dimensional constants
C_Lmax = 1.6  # stall CL
e = 0.92  # Oswald's efficiency factor
k = 1.17  # form factor
N_ult = 3.3  # ultimate load factor
S_wetratio = 2.075  # wetted area ratio
tau = 0.12  # airfoil thickness to chord ratio
W_W_coeff1 = 2e-5  # wing weight coefficient 1
W_W_coeff2 = 60  # wing weight coefficient 2

### Dimensional constants
Range = 1000e3  # aircraft range, m
TSFC = 0.6 / 3600  # thrust specific fuel consumption, 1/sec
V_min = 25  # takeoff speed, m/s
W_0 = 6250  # aircraft weight excluding wing, N


We then implement the free variables. Here, we do this basically the same way as SimPleAC.

As a side note: upon inspection, it's *very clear* that a lot of the quantities listed as "free variables" in the SimPleAC source are extraneous and can be explicitly computed. Examples:
* Reynolds number `Re` - there is no reason to declare this as a design variable when it can instead be explicitly computed as a function of cruise speed and wing geometry.
* Lift to drag ratio `LoD` - this is defined as a variable and then constrained to be equal to `L / D`, and then subsequently never used anywhere else in the problem formulation.
* `p_labor`, the labor cost, which is declared and then never used in the formulation.

Because of this, we remove a lot of the variable declarations, and solve the same engineering problem with **much** simpler notation.

We give initial guesses based on our intuition for order of magnitude. (Note for later readers: these initial guesses were made before looking at the optimal solution, in fairness.)

In [293]:
### Free variables (same as SimPleAC, with extraneous variables removed)
# LoD = opti.variable(init_guess=15)  # lift to drag ratio
# D = opti.variable(init_guess=10000 / 15, lower_bound=0)  # drag force, N
V = opti.variable(init_guess=50, log_transform=True)  # cruise speed, m/s
W = opti.variable(init_guess=10000, log_transform=True)  # total aircraft weight, N
# Re = opti.variable(init_guess=5e6)  # Reynolds number
# CDA0 = opti.variable(init_guess=0.02)  # Fuselage drag area, m^2
# C_D = opti.variable(init_guess=0.05)  # drag coefficient
C_L = opti.variable(init_guess=0.5)  # lift coefficient
# C_f = opti.variable(0.02)  # skin friction coefficient
W_f = opti.variable(init_guess=2000, lower_bound=0)  # fuel weight, N
# V_f = opti.variable(1)  # fuel volume, m^3
# V_f_avail = opti.variable(init_guess=1, lower_bound=0)  # fuel volume available, m^3
# T_flight = opti.variable(10 * 3600, lower_bound=0)  # flight time, sec

### More free variables
A = opti.variable(init_guess=15, lower_bound=0)  # aspect ratio
S = opti.variable(init_guess=20, log_transform=True)  # total wing area, m^2
# W_w = opti.variable(init_guess=1000, lower_bound=0)  # wing weight, N
# W_w_strc = opti.variable(init_guess=1000, lower_bound=0)  # wing structural weight, N
# W_w_surf = opti.variable(init_guess=1000, lower_bound=0)  # wing skin weight, N
# V_f_wing = opti.variable(init_guess=0.5)  # fuel volume in the wing, m^3
V_f_fuse = opti.variable(init_guess=0.0619, lower_bound=0)  # fuel volume in the fuselage, m^3

We implement the weight and lift models:

In [294]:
### Wing weight
W_w_surf = W_W_coeff2 * S
W_w_strc = W_W_coeff1 / tau * N_ult * A ** 1.5 * np.sqrt(
    (W_0 + V_f_fuse * g * rho_f) * W * S
)
W_w = W_w_surf + W_w_strc

### Entire weight
opti.subject_to(
    W >= W_0 + W_w + W_f
)

### Lift equals weight constraint
opti.subject_to([
    W_0 + W_w + 0.5 * W_f <= 0.5 * rho * S * C_L * V ** 2,
    W <= 0.5 * rho * S * C_Lmax * V_min ** 2,
])

### Flight duration
T_flight = Range / V

We implement the thrust and drag model:

In [295]:
### Drag
Re = (rho / mu) * V * (S / A) ** 0.5
C_f = 0.074 / Re ** 0.2

CDA0 = V_f_fuse / 10

C_D_fuse = CDA0 / S
C_D_wpar = k * C_f * S_wetratio
C_D_ind = C_L ** 2 / (np.pi * A * e)
C_D = C_D_fuse + C_D_wpar + C_D_ind
D = 0.5 * rho * S * C_D * V ** 2

opti.subject_to([
    W_f >= TSFC * T_flight * D,
])

[MX(fabs(opti36_lam_g_7))]

We implement the fuel volume model:

In [296]:
V_f = W_f / g / rho_f
V_f_wing = 0.03 * S ** 1.5 / A ** 0.5 * tau  # linear with b and tau, quadratic with chord

V_f_avail = V_f_wing + V_f_fuse

opti.subject_to(
    V_f_avail >= V_f
)

MX(fabs(opti36_lam_g_8))

We implement an objective, and we solve:

In [297]:
opti.minimize(W_f)

sol = opti.solve(max_iter=100)

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       27
Number of nonzeros in Lagrangian Hessian.............:       18

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        8
        inequality constraints with only lower bounds:        3
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        5

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 2

Let's print the value of our variables:

In [298]:
for value in [
    "V",
    "W",
    "C_L",
    "W_f",
    "A",
    "S",
    "V_f_fuse",
]:
    print(f"{value:10} = {opti.debug.value(eval(value)):.6}")


V          = 57.106
W          = 8704.82
C_L        = 0.290128
W_f        = 937.756
A          = 12.1049
S          = 14.1542
V_f_fuse   = 0.0619038


And we can print the value of some auxiliary parameters too:

In [299]:
for value in [
    "CDA0",
    "C_D",
    "C_L",
    "C_f",
    "D",
    "C_L/C_D",
    "Re",
    "T_flight",
    "V_f",
    "V_f_wing",
    "V_f_avail",
    "W_f",
    "W_w",
    "W_w_strc",
    "W_w_surf"
]:
    print(f"{value:10} = {opti.debug.value(eval(value)):.6}")

CDA0       = 0.00619038
C_D        = 0.0113188
C_L        = 0.290128
C_f        = 0.00349109
D          = 321.309
C_L/C_D    = 25.6325
Re         = 4.27908e+06
T_flight   = 17511.3
V_f        = 0.117003
V_f_wing   = 0.0550997
V_f_avail  = 0.117003
W_f        = 937.756
W_w        = 1517.06
W_w_strc   = 667.811
W_w_surf   = 849.25
